## Importing libreries:

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge #ordinary linear regression + w/ ridge regularization
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn import preprocessing
le = preprocessing.LabelEncoder()



In [2]:
#df = pd.read_csv('/Users/reema/Desktop/ML/vehicles.csv')

In [3]:
df = pd.read_csv('/Users/soaadaljafr/Downloads/vehicles.csv')

In [4]:
#df.transmission.value_counts()

## EDA
### Data Exploring:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [6]:
df.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


## Data Cleaning:

In [7]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [8]:
seed=60
df_train, df_test= train_test_split(df, test_size=0.2, random_state=seed)
df_train, df_val= train_test_split(df_train, test_size=0.2, random_state=seed)
df_train.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
323956,7316392177,https://portland.craigslist.org/clk/ctd/d/vanc...,portland,https://portland.craigslist.org,42996,2019.0,cadillac,xt4,NaN,NaN,...,NaN,NaN,NaN,https://images.craigslist.org/00p0p_dqtU24PjDG...,CARR Cadillac Buick GMC Vancouver address: ...,NaN,or,45.645296,-122.603772,2021-05-03T13:00:46-0700
305867,7311707558,https://oklahomacity.craigslist.org/ctd/d/okla...,oklahoma city,https://oklahomacity.craigslist.org,0,2007.0,ford,f150,NaN,NaN,...,NaN,other,NaN,https://images.craigslist.org/00S0S_arkVQYlVzC...,2007 FORD F150 SUPERCREW Offered by: Cars-...,NaN,ok,35.414450,-97.491710,2021-04-23T21:32:41-0500
347398,7313328718,https://greenville.craigslist.org/ctd/d/rock-h...,greenville / upstate,https://greenville.craigslist.org,4495,2008.0,saturn,astra,excellent,4 cylinders,...,compact,coupe,NaN,https://images.craigslist.org/01616_gnJBNrODqh...,2008 Saturn Astra Coupe Ready To Upgrade You...,NaN,sc,34.952829,-80.998136,2021-04-27T11:17:14-0400
75628,7316812372,https://denver.craigslist.org/cto/d/strasburg-...,denver,https://denver.craigslist.org,6495,1986.0,chevrolet,k5 blazer,good,8 cylinders,...,full-size,SUV,green,https://images.craigslist.org/00i0i_82rYlxPw9e...,Are you ready for the Apocalypse? You can be ...,NaN,co,39.698500,-104.201900,2021-05-04T11:27:36-0600
307728,7304786847,https://texoma.craigslist.org/ctd/d/ardmore-20...,texoma,https://texoma.craigslist.org,43700,2018.0,chevrolet,silverado 2500hd lt,excellent,8 cylinders,...,full-size,truck,white,https://images.craigslist.org/00T0T_kkWE6A7JIT...,2018 CHEVROLET SILVERADO 2500HD LT*WHITE*4X4*T...,NaN,ok,34.156600,-97.179200,2021-04-10T13:06:37-0500


In [9]:
print(df_train['county'].describe())
print(df_train['url'].describe())
print(df_train['region_url'].describe())
print(df_train['image_url'].describe())
print(df_train['description'].describe())
print(df_train['id'].describe())
print(df_train['VIN'].describe())

count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: county, dtype: float64
count                                                273203
unique                                               273203
top       https://athensga.craigslist.org/ctd/d/bmw-x5-a...
freq                                                      1
Name: url, dtype: object
count                           273203
unique                             413
top       https://omaha.craigslist.org
freq                              1974
Name: region_url, dtype: object
count                                                273156
unique                                               170404
top       https://images.craigslist.org/00N0N_1xMPvfxRAI...
freq                                                   4673
Name: image_url, dtype: object
count                                                273155
unique                                               238563
top       35 VEHICLES P

In [10]:
df_train.drop(['county','url','region_url','image_url','description','id','VIN'],axis=1, inplace=True)

df_val.drop(['county','url','region_url','image_url','description','id','VIN'],axis=1, inplace=True)

In [11]:
df_train.isna().sum()

region               0
price                0
year               765
manufacturer     11230
model             3327
condition       111671
cylinders       114088
fuel              1930
odometer          2819
title_status      5267
transmission      1623
drive            83704
size            196077
type             59575
paint_color      83531
state                0
lat               4222
long              4222
posting_date        47
dtype: int64

In [12]:
df_val.isna().sum()

region              0
price               0
year              212
manufacturer     2862
model             835
condition       27847
cylinders       28377
fuel              437
odometer          691
title_status     1259
transmission      397
drive           20925
size            48971
type            14830
paint_color     20627
state               0
lat              1021
long             1021
posting_date       10
dtype: int64

In [13]:
df_train.dropna(subset = ["year"], inplace=True)

In [14]:
df_val.dropna(subset = ["year"], inplace=True)

In [15]:
df_train.dropna(subset = ["manufacturer"], inplace=True)

In [16]:
df_val.dropna(subset = ["manufacturer"], inplace=True)

In [17]:
df_train['manufacturer'] = df_train['manufacturer'].replace(['lincoln','volvo','mitsubishi','mini','pontiac','rover','jaguar','porsche','mercury','saturn','alfa-romeo','tesla','fiat','harley-davidson','ferrari','datsun','aston-martin','land rover','morgan'],"other");

In [18]:
df_val['manufacturer'] = df_val['manufacturer'].replace(['lincoln','volvo','mitsubishi','mini','pontiac','rover','jaguar','porsche','mercury','saturn','alfa-romeo','tesla','fiat','harley-davidson','ferrari','datsun','aston-martin','land rover','morgan'],"other");

In [19]:
df_train.model.fillna('other',inplace=True)

In [20]:
df_val.model.fillna('other',inplace=True)

In [21]:
df_train.condition.fillna('good',inplace=True)

In [22]:
df_val.condition.fillna('good',inplace=True)

In [23]:
condition = df_train[["condition"]]

In [24]:
condition = df_val[["condition"]]

In [25]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
condition_encoded= ordinal_encoder.fit_transform(condition)
condition_encoded[:20]

array([[2.],
       [2.],
       [2.],
       [0.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.],
       [0.],
       [2.],
       [2.],
       [2.],
       [2.],
       [0.],
       [0.],
       [2.],
       [2.],
       [2.]])

In [26]:
type(df_train['condition'])

pandas.core.series.Series

In [27]:
type(df_val['condition'])

pandas.core.series.Series

In [28]:
df_train['cylinders'] = df_train['cylinders'].replace(['other','oth'],"6 cylinders");

In [29]:
df_train['cylinders'] = df_train.cylinders.str.strip(' cylinders')

In [30]:
df_train.cylinders.fillna('6',inplace=True)

In [31]:
df_train['cylinders']= df_train['cylinders'].astype(int)

In [32]:
df_val['cylinders'] = df_val['cylinders'].replace(['other','oth'],"6 cylinders");

df_val['cylinders'] = df_val.cylinders.str.strip(' cylinders')

df_val.cylinders.fillna('6',inplace=True)

df_val['cylinders']= df_val['cylinders'].astype(int)

In [33]:
df_train.fuel.value_counts() 

gas         220495
other        18765
diesel       16768
hybrid        3298
electric      1020
Name: fuel, dtype: int64

In [34]:
df_train.fuel.fillna(df_train.fuel.mode ,inplace=True)

In [35]:
df_val.fuel.fillna(df_train.fuel.mode ,inplace=True)

In [36]:
df_val.dropna(subset = ["odometer"], inplace=True)

In [37]:
df_train.dropna(subset = ["odometer"], inplace=True)

In [38]:
df_train.title_status.fillna('clean',inplace=True)

In [39]:
df_val.title_status.fillna('clean',inplace=True)

In [40]:
title_status = df_train[["title_status"]]

In [41]:
title_status = df_val[["title_status"]]

In [42]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
title_status_encoded= ordinal_encoder.fit_transform(title_status)

title_status_encoded[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [43]:
type(df_train['title_status'])

pandas.core.series.Series

In [44]:
df_train['transmission'].isna().sum()

1031

In [45]:
#df_train['transmission'] = df_train['transmission'].replace(['other'],"automatic");

In [46]:
#df_val['transmission'] = df_train['transmission'].replace(['other'],"automatic");

In [47]:
df_train.transmission.value_counts()

automatic    205630
other         38482
manual        14168
Name: transmission, dtype: int64

In [48]:
df_train.transmission.fillna(df_train.transmission.mode ,inplace=True)

In [49]:
df_val.transmission.fillna(df_train.transmission.mode ,inplace=True)

In [50]:
df_train.transmission.sample(5)

146962    automatic
149991    automatic
363091    automatic
250724    automatic
214568        other
Name: transmission, dtype: object

In [51]:
df_train.drive.fillna('other',inplace=True)

In [52]:
df_val.drive.fillna('other',inplace=True)

In [53]:
df_train.dropna(subset = ["paint_color"], inplace=True)

In [54]:
df_val.dropna(subset = ["paint_color"], inplace=True)

In [55]:
df_train.type.fillna('other',inplace=True)

In [56]:
df_val.type.fillna('other',inplace=True)

In [57]:
df_train['size'].fillna('other',inplace=True)

In [58]:
df_val['size'].fillna('other',inplace=True)

In [59]:
df_train.lat.fillna(45.645296,inplace=True)

In [60]:
df_val.lat.fillna(45.645296,inplace=True)

In [61]:
df_train.long.fillna(35.414450,inplace=True)

In [62]:
df_val.long.fillna(35.414450,inplace=True)

In [63]:
#df_train[['transmission','condition','title_status']] = df_train[['transmission','condition','title_status']].apply(le.fit_transform)


In [64]:
#df_val[['transmission','condition','title_status']] = df_val[['transmission','condition','title_status']].apply(le.fit_transform)


In [65]:
#feul = df_train[["fuel"]]

In [66]:
#feul1 = df_val[["fuel"]]

In [67]:
#feul = feul.apply(lambda col: le.fit_transform(col.astype(int)), axis=0, result_type='expand')

In [68]:
#feul1 = feul1.apply(lambda col: le.fit_transform(col.astype(int)), axis=0, result_type='expand')

In [69]:
#df_train[["fuel"]] = df_train[["fuel"]].apply(le.fit_transform)




In [70]:
#df_val = df_val.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

In [71]:
#df_val[["fuel"]] = df_val[["fuel"]].apply(le.fit_transform)



In [72]:
df_train = df_train.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

In [73]:
df_val = df_val.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

In [74]:
df_train.isna().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
lat             0
long            0
posting_date    0
dtype: int64

In [75]:
df_val.isna().sum()

region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
lat             0
long            0
posting_date    0
dtype: int64

In [76]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180809 entries, 75628 to 84716
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   region        180809 non-null  int64
 1   price         180809 non-null  int64
 2   year          180809 non-null  int64
 3   manufacturer  180809 non-null  int64
 4   model         180809 non-null  int64
 5   condition     180809 non-null  int64
 6   cylinders     180809 non-null  int64
 7   fuel          180809 non-null  int64
 8   odometer      180809 non-null  int64
 9   title_status  180809 non-null  int64
 10  transmission  180809 non-null  int64
 11  drive         180809 non-null  int64
 12  size          180809 non-null  int64
 13  type          180809 non-null  int64
 14  paint_color   180809 non-null  int64
 15  state         180809 non-null  int64
 16  lat           180809 non-null  int64
 17  long          180809 non-null  int64
 18  posting_date  180809 non-null  int64
dtyp

In [77]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45474 entries, 216319 to 259661
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   region        45474 non-null  int64
 1   price         45474 non-null  int64
 2   year          45474 non-null  int64
 3   manufacturer  45474 non-null  int64
 4   model         45474 non-null  int64
 5   condition     45474 non-null  int64
 6   cylinders     45474 non-null  int64
 7   fuel          45474 non-null  int64
 8   odometer      45474 non-null  int64
 9   title_status  45474 non-null  int64
 10  transmission  45474 non-null  int64
 11  drive         45474 non-null  int64
 12  size          45474 non-null  int64
 13  type          45474 non-null  int64
 14  paint_color   45474 non-null  int64
 15  state         45474 non-null  int64
 16  lat           45474 non-null  int64
 17  long          45474 non-null  int64
 18  posting_date  45474 non-null  int64
dtypes: int64(19)
memory

## Data Analysis and Visulizations:

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_train.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_val.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
#sns.pairplot(df_train, height=1.2, aspect=1.5);

In [ ]:
#sns.pairplot(df_val, height=1.2, aspect=1.5);

In [ ]:
y = df_train['price']
x = df_train['year']
plt.scatter(x, y)
plt.xlabel('year')
plt.ylabel('price')
plt.show()

In [ ]:
y = df_train['price']
x = df_train['odometer']
plt.scatter(x, y)
plt.xlabel('odometer')
plt.ylabel('price')
plt.show()

In [ ]:
sns.catplot(x="drive", y="price",kind="bar", data=df_train,ci=True);

In [ ]:
plt.figimage(20,20)
sns.catplot(x="manufacturer", y="price",kind="bar", data=df_train,ci=True);

## Feature Engineering:

Convert features to Dummy variables:

In [ ]:
plt.figure(figsize=(12,10))
cor = df_train.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#df_train["odometer"] = np.sqrt(preprocessing.minmax_scale(df_train["odometer"]))

In [79]:
df1 = df_train.select_dtypes(exclude=['object']).copy()
df_obj_1=df_train.loc[:,['fuel','size','transmission','condition','title_status','paint_color']]
#df_obj_2= df_train.loc[:,['fuel','transmission','paint_color']],
#df_obj_3= df_train.loc[:,['model']]

In [80]:
df_v = df_val.select_dtypes(exclude=['object']).copy()
df_obj_v=df_val.loc[:,['fuel','size','transmission','condition','title_status','paint_color']]
#df_obj_2= df_train.loc[:,['fuel','transmission','paint_color']],
#df_obj_3= df_train.loc[:,['model']]

In [81]:
df_dum_tr = pd.get_dummies(df_obj_1, drop_first=True)

cars_train1 = pd.concat([df1, df_dum_tr], axis=1)
cars_train1.head(2)

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,state,lat,long,posting_date,fuel,size,transmission,condition,title_status,paint_color
75628,80,8730,68,5,8222,2,6,1,62292,0,...,5,16541,251,162951,1,1,1,2,0,4
307728,358,7321,100,5,12008,0,6,1,57062,0,...,36,7079,30133,19617,1,1,1,0,0,10


In [82]:
df_dum_va = pd.get_dummies(df_obj_v, drop_first=True)

cars_val2 = pd.concat([df_v, df_dum_va], axis=1)
cars_val2.head(2)

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,...,state,lat,long,posting_date,fuel,size,transmission,condition,title_status,paint_color
216319,216,3280,75,6,6686,2,5,3,7025,0,...,23,12087,11428,33231,3,1,1,2,0,3
62863,0,3782,82,19,5597,2,4,3,3454,0,...,4,5159,2948,38396,3,3,1,2,0,10


In [ ]:
#df_dum_3 = pd.get_dummies(df_obj_3, drop_first=True)

In [ ]:
#cars_train3 = pd.concat([df1, df_dum_3], axis=1)
#cars_train3.head(2)

In [83]:
used_cars = pd.concat([df1,cars_val2,cars_train1], axis=1)

In [ ]:
#used_cars.info() 

## Linear Regression Models:

- Setting up for modeling and building a baseline:

In [84]:
X_train1 = cars_train1.drop(['price'], axis = 1)
y_train1 = cars_train1['price']

In [85]:
lr = LinearRegression()
lr.fit(X_train1,y_train1)

print(f"Training Score: {lr.score(X_train1,y_train1)}")

Training Score: 0.02584224860986295


In [86]:
X_val1 = cars_val2.drop(['price'], axis = 1)
y_val1 = cars_val2['price']

In [87]:
lr = LinearRegression()
lr.fit(X_val1,y_val1)

print(f"Training Score: {lr.score(X_val1,y_val1)}")

Training Score: 0.0260573394504795


In [ ]:
#from sklearn.tree import DecisionTreeRegressor
#dtree_reg = DecisionTreeRegressor()
#dtree_reg.fit(X_train1, y_train1)

#print(f"Training Score: {lr.score(X_train1,y_train1)}")

In [ ]:
#from sklearn.tree import DecisionTreeRegressor
#dtree_reg = DecisionTreeRegressor()
#dtree_reg.fit(X_val1, y_val1)

#print(f"Training Score: {lr.score(X_val1,y_val1)}")

In [ ]:
#from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor

In [ ]:
#rf_reg = RandomForestRegressor()
#rf_reg.fit(X_train1, y_train1)
#print(f"Training Score: {lr.score(X_train1,y_train1)}")

In [ ]:
#rf_reg = RandomForestRegressor()
#rf_reg.fit(X_val1, y_val1)
#print(f"Training Score: {lr.score(X_val1,y_val1)}")

## ______________________________________________________________________________________
### 1. Polynomial 

In [88]:
poly = PolynomialFeatures(degree=3, include_bias=False, interaction_only=False) 
X_train1_poly = poly.fit_transform(X_train1.values)
poly_model = LinearRegression()


In [89]:
poly_model.fit(X_train1_poly, y_train1)
print(poly_model.score(X_train1_poly, y_train1))

0.13902251758536877


In [90]:
scaler = StandardScaler()

X_train1 = scaler.fit_transform(X_train1 )

In [91]:
X_train1 

array([[-0.96763512, -3.04251696, -0.99754807, ...,  0.48009491,
        -0.17468555, -0.39366635],
       [ 1.38665008,  0.73506057, -0.99754807, ..., -1.62033411,
        -0.17468555,  1.05658176],
       [ 1.44593064,  0.61701127, -0.99754807, ...,  0.48009491,
        -0.17468555, -0.15195833],
       ...,
       [-1.34872445,  0.14481408, -1.45167672, ..., -1.62033411,
        -0.17468555, -1.36049842],
       [ 0.21797613,  0.97115916, -0.54341942, ...,  0.48009491,
        -0.17468555,  1.05658176],
       [-0.48492196,  0.14481408, -0.54341942, ...,  0.48009491,
        -0.17468555,  1.05658176]])

In [92]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train1)
X_train1_scaled = scaler.transform(X_train1)


In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.ensemble import AdaBoostRegressor

In [94]:
lrmodel = MLPRegressor()
lrmodel.fit(X_train1_scaled , y_train1)
print('Training data score',lrmodel.score(X_train1_scaled , y_train1));

/Users/soaadaljafr/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Training data score 0.1957457230813331


In [95]:
abmodel = AdaBoostRegressor()
abmodel.fit(X_train1 , y_train1)
print(abmodel.score(X_train1 , y_train1),'\n')

0.08572718720047012 



In [96]:
abmodel = AdaBoostRegressor()
abmodel.fit(X_train1 , y_train1)
print(abmodel.score(X_val1 , y_val1),'\n')

-0.011383212020657396 



## ______________________________________________________________________________________
### 2. Ridge


In [107]:
scaler = StandardScaler()

X_train1_scaled = scaler.fit_transform(X_train1)
X_val1_scaled = scaler.transform(X_val1)

red_model = Ridge(alpha=1)

In [108]:
red_model.fit(X_train1_scaled, y_train1)

print(f'Ridge Regression validation Score: {red_model.score(X_val1_scaled, y_val1):.8f}')
print(f'Ridge Regression Training Score: {red_model.score(X_train1_scaled, y_train1):.8f}')

Ridge Regression validation Score: -14556702.34702303
Ridge Regression Training Score: 0.02584225
